<a href="https://colab.research.google.com/github/shilz1007/shilz1007/blob/main/Sydney_Urban_Objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install trimesh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 663 kB 22.6 MB/s 


In [ ]:
import tensorflow as tf
import numpy as np
import tarfile
import os
import glob
import trimesh
from matplotlib import pyplot as plt
import pathlib

In [ ]:
import os
import glob
import trimesh
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,Model,activations
from tensorflow.keras.layers import Activation,Dense,Flatten,Conv1D,Dropout,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import Regularizer
from matplotlib import pyplot as plt

tf.random.set_seed(1234)

In [ ]:
path = 'file:///content/drive/MyDrive/sydney-urban-objects-dataset.tar.gz'

In [ ]:
path1 = 'https://www.acfr.usyd.edu.au/papers/data/sydney-urban-objects-dataset.tar.gz'

In [ ]:
Extracted_File = tf.keras.utils.get_file("Sidney_City_Data",path, archive_format='tar', untar=True)

79524613/79524613 [==============================] - 2s 0us/step


In [ ]:
Extracted_File

'/root/.keras/datasets/Sidney_City_Data'

In [ ]:
Extracted_File=pathlib.Path(Extracted_File)
batch_size = 32

In [ ]:
print(Extracted_File)
print(Extracted_File.glob('*'))

/root/.keras/datasets/Sidney_City_Data
<generator object Path.glob at 0x7f3d21b51650>


In [ ]:
for i in Extracted_File.glob('*'):
  print(i)

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  Extracted_File,
  validation_split=0.2,
  subset="training",
  seed=123,
  batch_size=batch_size,
  shuffle=False)

NotFoundError: ignored

In [ ]:
import os
base = os.path.dirname(Extracted_File)
!find $base -type d -print

/root/.keras/datasets
/root/.keras/datasets/sydney-urban-objects-dataset
/root/.keras/datasets/sydney-urban-objects-dataset/scans
/root/.keras/datasets/sydney-urban-objects-dataset/objects
/root/.keras/datasets/sydney-urban-objects-dataset/folds


In [ ]:
labels_dict = {'4wd': 0, 
              'bench': 1, 
              'bicycle': 2, 
              'biker': 3, 
              'building': 4, 
              'bus': 5, 
              'car': 6, 
              'cyclist': 7, 
              'excavator': 8, 
              'pedestrian': 9, 
              'pillar': 10, 
              'pole': 11, 
              'post': 12, 
              'scooter': 13, 
              'ticket_machine': 14, 
              'traffic_lights': 15, 
              'traffic_sign': 16, 
              'trailer': 17, 
              'trash': 18, 
              'tree': 19, 
              'truck': 20, 
              'trunk': 21, 
              'umbrella': 22, 
              'ute': 23, 
              'van': 24, 
              'vegetation': 25}

num_classes = len(labels_dict.keys())
num_classes

26

In [ ]:
train_data_list = []
train_label_list = []


In [ ]:
import glob
for i in glob.glob('/root/.keras/datasets/sydney-urban-objects-dataset/objects/*.csv'):
  XYZ = []
  ##print(i)
  label = os.path.basename(i).split('.')[0]
  ##print(label)
  train_label_list.append(labels_dict[label])
  with open(i) as csv_file:
    for j in csv_file.readlines():
      xyz_temp = j.split(',')
      XYZ.append([float(xyz_temp[3]),float(xyz_temp[4]),float(xyz_temp[5])])
      
  train_data_list.append(XYZ)    
##print(train_data_list)
print(train_label_list)  

[21, 19, 6, 21, 4, 2, 9, 16, 11, 9, 9, 11, 9, 20, 6, 11, 16, 24, 23, 9, 10, 9, 14, 2, 21, 6, 24, 6, 6, 9, 16, 11, 6, 21, 6, 16, 9, 23, 4, 9, 22, 9, 9, 9, 19, 5, 6, 19, 6, 9, 9, 20, 9, 16, 11, 10, 25, 19, 10, 11, 9, 5, 21, 0, 9, 16, 9, 19, 5, 9, 22, 6, 21, 9, 15, 11, 6, 6, 9, 9, 23, 17, 11, 21, 18, 9, 6, 24, 14, 3, 6, 15, 0, 21, 6, 10, 9, 6, 21, 9, 6, 15, 19, 9, 0, 9, 9, 9, 16, 15, 19, 6, 6, 9, 15, 9, 10, 21, 14, 9, 16, 7, 16, 19, 16, 19, 0, 15, 24, 11, 6, 15, 5, 9, 0, 23, 24, 16, 5, 6, 23, 22, 12, 9, 16, 0, 24, 6, 6, 24, 11, 10, 16, 15, 6, 6, 19, 2, 15, 16, 20, 6, 0, 16, 21, 15, 9, 9, 21, 9, 21, 20, 6, 6, 15, 21, 15, 9, 9, 9, 9, 9, 16, 16, 15, 9, 16, 9, 16, 9, 4, 12, 6, 16, 16, 6, 24, 9, 4, 9, 24, 15, 6, 24, 6, 4, 9, 4, 15, 6, 8, 9, 10, 15, 9, 12, 23, 20, 11, 21, 9, 6, 0, 24, 12, 9, 15, 24, 9, 9, 16, 9, 19, 4, 4, 9, 11, 9, 9, 9, 9, 16, 15, 16, 11, 21, 20, 20, 6, 16, 10, 6, 19, 6, 0, 21, 9, 21, 9, 9, 9, 15, 19, 16, 6, 21, 6, 15, 24, 16, 24, 15, 6, 15, 15, 6, 0, 23, 9, 7, 16, 15, 9, 19, 

In [ ]:
import glob
XYZ = []
row_len = []
for i in glob.glob('/root/.keras/datasets/sydney-urban-objects-dataset/objects/*.csv'):
  
  ##print(i)
  label = os.path.basename(i).split('.')[0]
  ##print(label)
  train_label_list.append(labels_dict[label])
  with open(i) as csv_file:
    csv_lines = csv_file.readlines()
    row_len.append(len(csv_lines))
    for j in csv_lines:
      xyz_temp = j.split(',')
      XYZ.append([float(xyz_temp[3]),float(xyz_temp[4]),float(xyz_temp[5])])
      
  #train_data_list.append(XYZ)    
##print(train_data_list)
train_data_list = XYZ
print(train_label_list) 

[21, 19, 6, 21, 4, 2, 9, 16, 11, 9, 9, 11, 9, 20, 6, 11, 16, 24, 23, 9, 10, 9, 14, 2, 21, 6, 24, 6, 6, 9, 16, 11, 6, 21, 6, 16, 9, 23, 4, 9, 22, 9, 9, 9, 19, 5, 6, 19, 6, 9, 9, 20, 9, 16, 11, 10, 25, 19, 10, 11, 9, 5, 21, 0, 9, 16, 9, 19, 5, 9, 22, 6, 21, 9, 15, 11, 6, 6, 9, 9, 23, 17, 11, 21, 18, 9, 6, 24, 14, 3, 6, 15, 0, 21, 6, 10, 9, 6, 21, 9, 6, 15, 19, 9, 0, 9, 9, 9, 16, 15, 19, 6, 6, 9, 15, 9, 10, 21, 14, 9, 16, 7, 16, 19, 16, 19, 0, 15, 24, 11, 6, 15, 5, 9, 0, 23, 24, 16, 5, 6, 23, 22, 12, 9, 16, 0, 24, 6, 6, 24, 11, 10, 16, 15, 6, 6, 19, 2, 15, 16, 20, 6, 0, 16, 21, 15, 9, 9, 21, 9, 21, 20, 6, 6, 15, 21, 15, 9, 9, 9, 9, 9, 16, 16, 15, 9, 16, 9, 16, 9, 4, 12, 6, 16, 16, 6, 24, 9, 4, 9, 24, 15, 6, 24, 6, 4, 9, 4, 15, 6, 8, 9, 10, 15, 9, 12, 23, 20, 11, 21, 9, 6, 0, 24, 12, 9, 15, 24, 9, 9, 16, 9, 19, 4, 4, 9, 11, 9, 9, 9, 9, 16, 15, 16, 11, 21, 20, 20, 6, 16, 10, 6, 19, 6, 0, 21, 9, 21, 9, 9, 9, 15, 19, 16, 6, 21, 6, 15, 24, 16, 24, 15, 6, 15, 15, 6, 0, 23, 9, 7, 16, 15, 9, 19, 

In [ ]:
data = np.array(train_data_list)
label = np.array(train_label_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [ ]:
print(np.array(train_data_list[0]).shape)
print(data[:5])

(79, 3)
[list([[10.6959, 7.48081, 1.00437], [10.7055, 7.52285, 0.102211], [10.5869, 7.50399, 0.997741], [10.5995, 7.54812, 0.10089], [10.7754, 7.46112, 1.46073], [10.7987, 7.50574, 0.562702], [10.6567, 7.62935, 1.00908], [10.7288, 7.51231, -0.332226], [10.6335, 7.6483, 0.102468], [10.6277, 7.45781, 1.44571], [10.6061, 7.47085, 0.553231], [10.5804, 7.50766, -0.330386], [10.6434, 7.59782, 0.558764], [10.6255, 7.64068, -0.332078], [10.7402, 7.51311, 0.942297], [10.7556, 7.56918, 0.0295309], [10.613, 7.52408, 0.934244], [10.6182, 7.57288, 0.028131], [10.7527, 7.48443, 1.38624], [10.6877, 7.421, 0.482253], [10.6439, 7.62149, 0.940982], [10.7412, 7.50927, -0.409929], [10.6425, 7.66599, 0.0290727], [10.6851, 7.51898, 0.484824], [10.6448, 7.54201, -0.408684], [10.649, 7.59406, 0.48549], [10.6216, 7.6262, -0.409307], [10.7086, 7.47815, 0.865442], [10.7154, 7.51648, -0.0203721], [10.5801, 7.48756, 0.85778], [10.6403, 7.56392, -0.0206772], [10.7662, 7.42295, 1.33203], [10.7741, 7.46669, 0.425976]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data_list, train_label_list, test_size=0.20, random_state=42)

In [ ]:
batch_size = 32
print(X_train)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

AttributeError: ignored

In [ ]:
X_train_ds = tf.data.Dataset.from_generator(lambda: X_train, 
                                         tf.float32,
                                         tf.TensorShape([None, 3]))

In [ ]:
X_test_ds = tf.data.Dataset.from_generator(lambda: X_test, 
                                         tf.float32,
                                         tf.TensorShape([None, 3]))
y_train_ds = tf.data.Dataset.from_generator(lambda: y_train, 
                                         tf.float32,
                                         tf.TensorShape([]))
y_test_ds = tf.data.Dataset.from_generator(lambda: y_test, 
                                         tf.float32,
                                         tf.TensorShape([]))

In [ ]:
X_train_ds = tf.ragged.constant(X_train)
X_test_ds = tf.ragged.constant(X_test)
y_train_ds = tf.ragged.constant(y_train)
y_test_ds = tf.ragged.constant(y_test)

print(X_train_ds.shape)

                                        

(1009, None, None)


In [ ]:
X_train_ds = tf.RaggedTensor.from_row_lengths(train_data_list,row_len)


print(X_train_ds.shape)

(631, None, 3)


In [ ]:
for data in dataset:
  print(data)
  break

In [ ]:
def augment(points,labels):
  points += tf.random.uniform(points.shape,-0.005,0.005,dtype=tf.float64)
  points = tf.random.shuffle(points)
  return points,labels


train_dataset = tf.data.Dataset.zip((X_train,y_train))
test_dataset =  tf.data.Dataset.zip((X_test,y_test))

train_dataset = train_dataset.shuffle(len(X_train)).map(augment).batch(batch_size)
test_dataset = test_dataset.shuffle(len(X_test)).batch(batch_size)

TypeError: ignored

In [ ]:
def conv_layer(x,filters):
  x = layers.Conv1D(filters,kernel_size=1,padding='valid')(x)
  x = layers.BatchNormalization(momentum=0.0)(x)
  return layers.Activation('relu')(x)

def dense_layers(x,filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

In [ ]:
class OrthogonalRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

In [ ]:
def tnet(inputs, num_features):

    # Initalise bias as the indentity matrix
    bias = tf.keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_layer(inputs, 32)
    x = conv_layer(x, 64)
    x = conv_layer(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_layers(x, 256)
    x = dense_layers(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [ ]:
inputs = tf.keras.Input(shape=(points, 3))

x = tnet(inputs, 3)
x = conv_layer(x, 32)
x = conv_layer(x, 32)
x = tnet(x, 32)
x = conv_layer(x, 32)
x = conv_layer(x, 64)
x = conv_layer(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_layers(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_layers(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(classes, activation="softmax")(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
model.summary()

Model: "pointnet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 2048, 3)]    0           []                               
                                                                                                  
 conv1d_3 (Conv1D)              (None, 2048, 32)     128         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 2048, 32)    128         ['conv1d_3[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 activation_3 (Activation)      (None, 2048, 32)     0           ['batch_normalization_3[0]

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)



In [ ]:
#model.fit(train_dataset, epochs=20, validation_data=test_dataset)
model.fit(train_dataset,epochs=50,validation_data=test_dataset)

Epoch 1/50
125/125 [==============================] - 13s 61ms/step - loss: 3.6395 - sparse_categorical_accuracy: 0.2588 - val_loss: 6.5976 - val_sparse_categorical_accuracy: 0.1233
Epoch 2/50
125/125 [==============================] - 7s 55ms/step - loss: 3.0828 - sparse_categorical_accuracy: 0.3623 - val_loss: 287.5584 - val_sparse_categorical_accuracy: 0.1112
Epoch 3/50
125/125 [==============================] - 7s 54ms/step - loss: 2.8822 - sparse_categorical_accuracy: 0.4235 - val_loss: 4845082247168.0000 - val_sparse_categorical_accuracy: 0.3194
Epoch 4/50
125/125 [==============================] - 7s 54ms/step - loss: 2.7666 - sparse_categorical_accuracy: 0.4741 - val_loss: 14883110990765875003392.0000 - val_sparse_categorical_accuracy: 0.3381
Epoch 5/50
125/125 [==============================] - 7s 54ms/step - loss: 2.5897 - sparse_categorical_accuracy: 0.5342 - val_loss: 1415983474409472.0000 - val_sparse_categorical_accuracy: 0.5363
Epoch 6/50
125/125 [=======================

In [ ]:
X,y = 

In [ ]:
path_fold = '/root/.keras/datasets/sydney-urban-objects-dataset/folds/fold0.txt'

In [ ]:
with open(path_fold,'r') as f:
    lines = f.readlines()
    print(lines)

['4wd.0.2299.bin\n', '4wd.1.2446.bin\n', '4wd.13.270.bin\n', '4wd.6.16217.bin\n', '4wd.7.16217.bin\n', 'building.0.16217.bin\n', 'building.1.16217.bin\n', 'building.10.2446.bin\n', 'building.8.2299.bin\n', 'building.11.2446.bin\n', 'bus.11.270.bin\n', 'bus.12.270.bin\n', 'bus.7.2299.bin\n', 'bus.3.23124.bin\n', 'bus.4.23124.bin\n', 'car.62.2299.bin\n', 'car.67.270.bin\n', 'car.68.270.bin\n', 'car.63.3582.bin\n', 'car.65.3725.bin\n', 'car.66.4043.bin\n', 'car.69.5100.bin\n', 'car.70.5100.bin\n', 'car.71.5229.bin\n', 'car.80.12346.bin\n', 'car.81.12346.bin\n', 'car.82.12346.bin\n', 'car.83.12346.bin\n', 'car.84.12715.bin\n', 'car.85.12715.bin\n', 'car.86.12715.bin\n', 'car.87.12715.bin\n', 'car.22.17038.bin\n', 'car.23.17038.bin\n', 'car.25.17038.bin\n', 'car.26.17038.bin\n', 'car.29.17038.bin\n', 'car.30.17038.bin\n', 'pedestrian.100.270.bin\n', 'pedestrian.101.270.bin\n', 'pedestrian.102.270.bin\n', 'pedestrian.103.270.bin\n', 'pedestrian.104.270.bin\n', 'pedestrian.105.270.bin\n', 'pe

In [ ]:
folders = glob.glob('/root/.keras/datasets/sydney-urban-objects-dataset/objects/bin')
#files = glob.glob(folders[0])
print(folders[0])
#print(files)

IndexError: ignored

In [ ]:
files[0]

'/root/.keras/datasets/sydney-urban-objects-dataset/objects/pedestrian.63.2446.bin.meta'

In [ ]:

labels = []
sample_size =  50

In [ ]:
file_name = files[0].split("/")[-1]
final = file_name.split(".")[-2]
label = file_name.split(".")[0]
print(file_name)
print(final)
print(label)

pedestrian.63.2446.bin.meta
bin
pedestrian


In [ ]:

for file in files[0]:

    #print(file)
    file_name = files[0].split("/")[-1]
    print(file_name)
    example = file_name.split(".")[-2]
    print(example)
    if file_name.split(".")[-1] == "bin":
       #print('inside')
       label = file_name.split(".")[0] 
       #print(label)
       labels.append(labels_dict[label])
       files.append(file)
    
#print(files)      

pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestrian.63.2446.bin.meta
bin
pedestri

In [ ]:
import pandas as pd

In [ ]:
##df = pd.read_csv('/root/.keras/datasets/sydney-urban-objects-dataset/objects/4wd.0.2299.csv',encoding='ISO-8859-1')
try:
  df = pd.read_csv('/root/.keras/datasets/sydney-urban-objects-dataset/objects/4wd.0.2299.csv',encoding='ISO-8859-1')
  print('csv file has header.....')
except:
  print('csv file has no header.....')  

csv file has header.....


In [ ]:
df.head()

,20111104T013210.881031,1,56,6.21931,2.71008,1.14817,0.413941,6.88049,1841
0,20111104T013210.881033,17,59,6.43505,2.59084,1.50079,0.379826,7.09740,1845
1,20111104T013210.881036,1,32,5.81078,2.59336,2.49100,0.422970,6.83331,1851
2,20111104T013210.881054,1,56,6.12841,2.71615,1.13409,0.420224,6.79849,1881
3,20111104T013210.881056,1,59,6.19975,2.54224,1.44850,0.386110,6.85540,1886
4,20111104T013210.881069,1,45,6.46073,2.53374,1.88132,0.370813,7.19018,1905
